In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import  TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
path='/content/gdrive/MyDrive/Datasets/'
movies_df= pd.read_csv(path+'movies.csv', dtype={'movieId': 'int32', 'title': 'str'})
ratings_df= pd.read_csv(path+'ratings.csv',usecols=['userId', 'movieId', 'rating'],
            dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})

In [ ]:
tmdb_path='/content/gdrive/MyDrive/tmdb_movies.csv'
tmdb=pd.read_csv(tmdb_path)
tmdb

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""

In [ ]:

movies_df['title']= movies_df['title'].str.rstrip('0123456789() ')
movies_df


,movieId,title,genres
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji,Adventure|Children|Fantasy
2,3,Grumpier Old Men,Comedy|Romance
3,4,Waiting to Exhale,Comedy|Drama|Romance
4,5,Father of the Bride Part II,Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic,Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero,Animation|Comedy|Fantasy
9739,193585,Flint,Drama
9740,193587,Bungo Stray Dogs: Dead Apple,Action|Animation


In [ ]:
ratings_df.head()


,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [ ]:
df = pd.merge(ratings_df,movies_df,on='movieId')
df.head()

,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story,Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,Toy Story,Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,Toy Story,Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,Toy Story,Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,Toy Story,Adventure|Animation|Children|Comedy|Fantasy


In [ ]:
df['rating'] = df['rating'].fillna(0)

In [ ]:
# Create a TF-IDF matrix for movie genres
tfidf= TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies_df['genres'])

# Calculate cosine similarity
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Create an index mapping movie titles to movie indices
indices = pd.Series(movies_df.index, index=movies_df['title']).drop_duplicates()



In [ ]:
def get_content_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = enumerate(cosine_sim[idx])
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    sim_index = [i[0] for i in sim_scores]
    recommended_movies = movies_df['title'].iloc[sim_index]

    # Print only the movie names, not their indices
    recommended_movie_names = recommended_movies.values.tolist()
    return recommended_movie_names




# Example usage
title='Tangled'
content_based_recommended_movies= get_content_recommendations(title)
print("Recommended Movies:")

for idx, movie in enumerate(content_based_recommended_movies, start=1):
        print(f"{idx}. {movie}")



Recommended Movies:
1. Beauty and the Beast
2. Fantasia
3. Dr. Seuss' The Lorax
4. Aladdin and the King of Thieves
5. Cinderella
6. Princess and the Frog, The
7. Cloudy with a Chance of Meatballs
8. Christmas Carol, A
9. Frosty the Snowman
10. Enchanted


# Making App

In [ ]:
import pickle

In [ ]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 92.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.1 MB/s eta 0:00:00


In [ ]:
pickle.dump(movies_df.to_dict(),open('movies.pkl','wb'))
pickle.dump(cosine_sim ,open('cosine_sim.pkl','wb'))
pickle.dump(tmdb,open('tmdb.pkl','wb'))


In [ ]:
%%writefile app.py
import streamlit as st
import pickle
import pandas as pd
import numpy as np
import requests

#Load movies_dataframe

movies_dict= pickle.load(open('movies.pkl','rb'))

#Load cosine_similarity
cosine_sim= pickle.load(open('cosine_sim.pkl','rb'))

#Load tmdb datasets

tmdb= pickle.load(open('tmdb.pkl','rb'))


movies=pd.DataFrame(movies_dict)

# Create an index mapping movie titles to movie indices
indices = pd.Series(movies.index, index=movies['title']).drop_duplicates()



def fetch_poster(movie_id):
    url = "https://api.themoviedb.org/3/movie/{}?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US".format(movie_id)
    data = requests.get(url)
    data = data.json()
    poster_path = data['poster_path']
    full_path = "https://image.tmdb.org/t/p/w500/" + poster_path
    return full_path


def get_content_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the input movie title
    idx = indices[title]

    # Get the cosine similarity scores for the input movie
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_indices = np.argsort(sim_scores)[::-1][1:11]
    recommended_movie_titles = movies_df['title'].iloc[sim_indices].tolist()


    recommended_poster=[]

    for i in recommended_movie_names:
        id= (tmdb[tmdb['original_title']==i]['id']).values[0]
        recommended_poster.append(fetch_poster(id))



    return recommended_movie_titles,recommended_poster




st.title('Movie Recommendation System')

selected_movie_name = st.selectbox(
'Select movies',
movies['title'].values)

st.write('You selected:', selected_movie_name)

if st.button('Recommendaions are:'):
   recommended_movie_names,poster = get_content_recommendations(selected_movie_name)






Overwriting app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

[..................] \ rollbackFailedOptional: verb npm-session 9aa371c354ff2e5


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.225.199.240:8501

npx: installed 22 in 3.463s
your url is: https://ripe-ravens-wave.loca.lt
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)
2023-10-16 03:15:15.873 Uncaught app exception
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/streamlit/runtime/scriptrunner/script_runner.py", line 541, in _run_script
    exec(code, module.__dict__)
  File "/content/app.py", line 67, in <module>
    recommended_movie_names,p